In [1]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 3.3 MB/s 
     |████████████████████████████████| 67 kB 2.7 MB/s 
     |████████████████████████████████| 895 kB 35.7 MB/s 
     |████████████████████████████████| 596 kB 13.0 MB/s 
     |████████████████████████████████| 6.8 MB 39.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
# Import Important Libraries
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf 

In [6]:
#Import the dataframes

train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")
 

In [7]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# Now we can drop the useless columns 

train_df=train_df.drop(['id','keyword','location'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
X=train_df.text.values
y=train_df.target.values

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42,stratify=y)

In [12]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


tr_token=tokenizer(list(X_train), return_tensors='np',truncation=True, padding='longest') # padding="max_length"
val_token=tokenizer(list(X_val), return_tensors='np',truncation=True, padding='longest')# padding="max_length"

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [13]:
tr_token

{'input_ids': array([[  101,  2668,  2177, ...,     0,     0,     0],
       [  101,  2425,  1996, ...,     0,     0,     0],
       [  101,  1030, 24646, ...,     0,     0,     0],
       ...,
       [  101,  1001,  1052, ...,     0,     0,     0],
       [  101,  9964,  1064, ...,     0,     0,     0],
       [  101,  1030,  2984, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [14]:
val_token

{'input_ids': array([[  101,  1045,  5993, ...,     0,     0,     0],
       [  101,  1030, 21592, ...,     0,     0,     0],
       [  101,  2469,  1996, ...,     0,     0,     0],
       ...,
       [  101,  3040,  2692, ...,     0,     0,     0],
       [  101,  8112, 18806, ...,     0,     0,     0],
       [  101,  2123,  1005, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [15]:
# Convert to tensor datasets
train = tf.data.Dataset.from_tensor_slices((dict(tr_token),y_train))

valid = tf.data.Dataset.from_tensor_slices((dict(val_token),y_val))

In [16]:
# Shuffle th observations every time we use this object

train_shuffle = train.shuffle(buffer_size=len(X_train))

valid_shuffle = valid.shuffle(buffer_size=len(X_val))

In [19]:
# Now we are ready to form our batches, let's use the .batch method

train_batch = train_shuffle.batch(batch_size=32)

val_batch = valid_shuffle.batch(batch_size=32)

In [20]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [24]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 32
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(X_train) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [25]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer=opt,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],)


In [26]:
model.fit(
    train_batch,
    epochs=3,
    validation_data=val_batch,
)

Epoch 1/3
160/160 [==============================] - 5070s 32s/step - loss: 0.4588 - accuracy: 0.7947 - val_loss: 0.4204 - val_accuracy: 0.8380
Epoch 2/3
160/160 [==============================] - 4966s 31s/step - loss: 0.3027 - accuracy: 0.8831 - val_loss: 0.4034 - val_accuracy: 0.8372
Epoch 3/3
160/160 [==============================] - 4986s 31s/step - loss: 0.1910 - accuracy: 0.9325 - val_loss: 0.4668 - val_accuracy: 0.8408


In [40]:
model.save_weights("/content/mod")

In [30]:
X_test=test_df.text.values

In [45]:
X_test[0]

'Just happened a terrible car crash'

In [46]:
input_text_tokenized = tokenizer.encode(X_test[0],
                                            truncation=True,
                                            padding=True,
                                            return_tensors="tf")


In [51]:
import numpy as np

In [54]:
pred_list=[]
for i in X_test:
  input_text_tokenized = tokenizer.encode(i,
                                            truncation=True,
                                            padding=True,
                                            return_tensors="tf")
  preds = model.predict(input_text_tokenized)["logits"]
  class_preds = np.argmax(preds, axis=1)
  pred_list.append(class_preds[0])
 

In [56]:
test_df["predictions"]=pred_list

In [58]:
test_df

,id,keyword,location,text,predictions
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,1
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,1
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [59]:
test_df.to_excel("test_df.xlsx")